# 中華職棒2020上半季逐球資料整理

打者資料整理：
1. 關鍵時刻（至少追平分在2壘）打擊率、全壘打、長打
2. 球數絕對落後打擊率(2好0壞 )
3. 盜壘時機（幾好幾壞幾出局）

投手資料整理：
1. 球數絕對領先下一顆位置、好壞球
2. 每個投手對決時刻常用球種、進壘位置（2好2壞)
3. 後援連續出賽成績（有休息、連二、連三）

In [1]:
import pandas as pd
import math

In [8]:
df = pd.read_excel('../OpenData/2020中華職棒上半季-電子紀錄程式檔-revised-20200925.xlsx', sheet_name='工作表1')

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

# 打者資料整理

關鍵時刻（至少追平分在2壘）打擊率、全壘打、長打

In [11]:
df = df.loc[:,['主隊得分','客隊得分','局數','Inning Half','投手','捕手','投手球數','好球數','壞球數','出局數','球速',
 '球種','打者','揮棒','結果', '一壘跑者','二壘跑者','三壘跑者','得分者','非自責分','自責分','一出局','二出局','三出局','出局','安打',
 '失誤',]]
df.iloc[:,15:18] = df.iloc[:,15:18].astype('str')
df.iloc[:,23] = df.iloc[:,23].astype('str')

In [24]:
%%time
# 客隊挑出關鍵時刻的打席逐球
c = 0
KeyMoments = pd.DataFrame(columns=list(df.columns))
for i in range(len(df)):
    if df.iloc[i,3] == 'top':
        score = df.iloc[i-1,0] - df.iloc[i-1,1]
    else:
        score = df.iloc[i-1,1] - df.iloc[i-1,0]
    base = 0
    if df.iloc[i-1,16] != 'nan':
        base += 1
    if df.iloc[i-1,17] != 'nan':
        base += 1        
    if (score + base) >= 0 and score <= 0 and base != 0 and df.iloc[i - 1,23] == 'nan' and df.iloc[i-1,3] == df.iloc[i,3]:
        c +=1
        KeyMoments = KeyMoments.append(df.iloc[i,:])

CPU times: user 33.6 s, sys: 160 ms, total: 33.7 s
Wall time: 33.8 s


In [29]:
# 留下該打席結果
AtBat = pd.DataFrame(columns=list(KeyMoments.columns))
for i in range(len(KeyMoments) - 1):
    if KeyMoments.iloc[i, 12] != KeyMoments.iloc[i+1, 12]:
        AtBat = AtBat.append(KeyMoments.iloc[i, :])

In [30]:
AtBat = AtBat[AtBat['結果'].isnull() == False]

In [31]:
# 查看結果有那些記錄方式
name = []
for i in AtBat['結果']:
    if i not in name:
        name.append(i)
name

['四壞',
 '三振',
 '揮空三振',
 '接殺',
 '刺殺',
 '界飛接殺',
 '一壘安',
 '二壘安',
 '野手選擇',
 '雙殺打',
 '敬遠',
 '失誤',
 '全壘打',
 '高飛犧牲',
 '犧牲觸擊',
 '三壘安',
 '暴投',
 '觸身球',
 '投手犯規',
 '捕逸',
 '揮空',
 '高飛接殺',
 '平飛被接',
 '雙殺守備',
 '牽制出局',
 '不死三振']

In [32]:
AtBat = AtBat.loc[:,['打者','得分者','出局','結果']]

In [36]:
# '打者':['打席','安打','全雷打數','長打數']
statistics = {}
for i in range(len(AtBat)):
    if AtBat.iloc[i,0] not in statistics.keys():
        statistics[AtBat.iloc[i,0]] = [0,0,0,0,0]
    if AtBat.iloc[i,3] not in ['四壞', '失誤', '野手選擇','犧牲觸擊','高飛犧牲','觸身球','敬遠','牽制出局','揮空','捕逸','投手犯規','暴投']:
        if AtBat.iloc[i,3] == '全壘打':
            statistics[AtBat.iloc[i,0]][2] += 1
        if AtBat.iloc[i,3] in ['全壘打', '二壘安', '三壘安']:
            statistics[AtBat.iloc[i,0]][3] += 1
        if AtBat.iloc[i,3] in ['全壘打', '二壘安', '三壘安', '一壘安']:
            statistics[AtBat.iloc[i,0]][1] += 1
        statistics[AtBat.iloc[i,0]][0] += 1
        

In [37]:
for i in statistics:
    if statistics[i][0] != 0:
        statistics[i][4] = round((statistics[i][1] / statistics[i][0]),3)

In [54]:
df = pd.DataFrame(statistics).T
df.columns = ['打席','安打','全壘打數','長打數','打擊率']

In [58]:
df[df['打席'] >= 10].sort_values(by='打擊率', ascending=False)

,打席,安打,全壘打數,長打數,打擊率
統一陳傑憲,14.0,8.0,0.0,3.0,0.571
富邦范國宸,18.0,9.0,0.0,1.0,0.500
兄弟周思齊,15.0,7.0,1.0,5.0,0.467
富邦高國麟,15.0,7.0,0.0,3.0,0.467
統一吳桀睿,11.0,5.0,0.0,1.0,0.455
...,...,...,...,...,...
富邦林益全,28.0,6.0,0.0,1.0,0.214
統一郭阜林,10.0,2.0,0.0,2.0,0.200
樂天林承飛,11.0,2.0,0.0,1.0,0.182
兄弟王威晨,12.0,2.0,0.0,0.0,0.167
